## Optimal control with CasADi

### Define model and cost

In [ ]:
import numpy as np
import casadi as ca


# cartpole class
class CartPole:
    def __init__(self, module='casadi'):
        if module == 'casadi':
            self.sin = ca.sin
            self.cos = ca.cos
            self.pi = ca.pi
        elif module == 'numpy':
            self.sin = np.sin
            self.cos = np.cos
            self.pi = np.pi
        else:
            raise TypeError

        self.mc = 2.0
        self.mp = 0.2
        self.l = 0.5
        self.gc = 9.80665

        self.lbu = -15.0
        self.ubu = 15.0
        self.lby = 1.0
        self.uby = 1.0

    def dynamics(self, x, u):
        sin = self.sin
        cos = self.cos
        pi = self.pi

        mc = self.mc
        mp = self.mp
        l = self.l
        gc = self.gc

        y = x[0]
        th = x[1]
        dy = x[2]
        dth = x[3]
        f = u[0]

        ddy = (f+mp*sin(th)*(l*dth*dth+gc*cos(th))) / (mc+mp*sin(th)*sin(th)) 
        ddth = (-f*cos(th)-mp*l*dth*dth*cos(th)*sin(th)-(mc+mp)*gc*sin(th)) / (l * (mc+mp*sin(th)*sin(th))) 
        return np.array([dy, dth, ddy, ddth])


In [ ]:
# cost calculating class
class CostFunction:
    def __init__(self, module='casadi'):
        if module == 'casadi':
            self.sin = ca.sin
            self.cos = ca.cos
            self.pi = ca.pi
        elif module == 'numpy':
            self.sin = np.sin
            self.cos = np.cos
            self.pi = np.pi
        else:
            raise TypeError
        pi = self.pi

        self.n_x = 4
        self.n_u = 1
        self.x_ref = np.array([0.0, pi, 0.0, 0.0])
        self.Q = np.array([2.5, 10.0, 0.01, 0.01])
        self.R = np.array([1.0])
        self.Q_f = np.array([2.5, 10.0, 0.01, 0.01])

    def stage_cost(self, x, u):
        l = 0.0
        for i in range(self.n_x):
            l += 0.5 * self.Q[i] * (x[i] - self.x_ref[i]) ** 2
        for i in range(self.n_u):
            l += 0.5 * self.R[i] * u[i]**2
        return l
    
    def terminal_cost(self, x):
        lf = 0.0
        for i in range(self.n_x):
            lf += 0.5 * self.Q_f[i] * (x[i] - self.x_ref[i]) ** 2
        return lf

    

In [ ]:
cartpole = CartPole()
cost = CostFunction()


### Approximate optimal control problem and formulate it as NLP

In [ ]:
# opti interface
opti = ca.Opti()

# dimension of state and input
n_x = 4
n_u = 1

# horizon length[s], total grids
T = 5.0
N = 200
dt = T / N

# decision variables (state, input) over the entire horizon.
xs = opti.variable(n_x, N + 1)
us = opti.variable(n_u, N)
opti.set_initial(xs, np.zeros((n_x, N + 1)))
opti.set_initial(us, np.zeros((n_u, N)))

# initial state
x0 = opti.parameter(n_x)
opti.set_value(x0, np.zeros(n_x))

# x0 = np.zeros(n_x)

In [ ]:
# cost
J = 0.0
for i in range(N):
    J += cost.stage_cost(xs[:, i], us[:, i]) * dt
J += cost.terminal_cost(xs[:, N])

opti.minimize(J)

# state space equasion as equality constraints
opti.subject_to(xs[:, 0] == x0)
for i in range(N):
    f_array = cartpole.dynamics(xs[:, i], us[:, i])
    f = ca.vertcat(*f_array)
    x1 = xs[:, i] + f * dt
    opti.subject_to(xs[:, i + 1] == x1)

# bound for control input
for i in range(N):
    opti.subject_to(opti.bounded(cartpole.lbu, us[:, i], cartpole.ubu))
    # opti.subject_to(cartpole.lbu <= us[:, i])
    # opti.subject_to(us[:, i] <= cartpole.ubu)


In [ ]:
# not working
# cost_hist = []
# opti.callback(lambda i: cost_hist.append(opti.debug.value(J)))

### Choose solver and solve NLP

In [ ]:
# use IPOPT as NLP solver
solver_option = {'print_time': True, 'calc_lam_x': True, 'calc_lam_p': True, 'ipopt': {'mu_min': 0.001, 'max_iter': 1000, 'warm_start_init_point': 'yes', 'print_level':5, 'print_timing_statistics':'no'}}
opti.solver('ipopt', solver_option)

# solve
try:
    sol = opti.solve()
except RuntimeError:
    print('\n[ERROR IN SOLVE]')


### Plot trajectory

In [ ]:
import path_ocpy
from ocpy.plotter import Plotter


# xs_opt = sol.value(xs).T
# us_opt = sol.value(us).T
xs_opt = opti.debug.value(xs).T
us_opt = opti.debug.value(us).T
ts = np.array([i*dt for i in range(N + 1)])

log_dir = path_ocpy.dir + '../log/cartpole'

# print('final cost:', sol.value(J))
print('final cost:', opti.debug.value(J))

plotter = Plotter(log_dir, xs_opt, us_opt, ts)
plotter.plot(save=False)


## Model predictive control with CasADi

### MPC class

In [ ]:
class MPC:
    def __init__(self):
        # opti interface
        opti = ca.Opti()

        # dimension of state and input
        n_x = 4
        n_u = 1

        # horizon length[s], total grids
        T = 2.0
        N = 40
        dt = T / N

        # decision variables (state, input) over the entire horizon.
        xs = opti.variable(n_x, N + 1)
        us = opti.variable(n_u, N)
        opti.set_initial(xs, np.zeros((n_x, N + 1)))
        opti.set_initial(us, np.zeros((n_u, N)))

        # initial state
        x0 = opti.parameter(n_x)
        opti.set_value(x0, np.zeros(n_x))

        # cost
        J = 0.0
        for i in range(N):
            J += cost.stage_cost(xs[:, i], us[:, i]) * dt
        J += cost.terminal_cost(xs[:, N])

        opti.minimize(J)

        # state space equasion as equality constraints
        opti.subject_to(xs[:, 0] == x0)
        for i in range(N):
            f_array = cartpole.dynamics(xs[:, i], us[:, i])
            f = ca.vertcat(*f_array)
            x1 = xs[:, i] + f * dt
            opti.subject_to(xs[:, i + 1] == x1)

        # bound for control input
        for i in range(N):
            opti.subject_to(opti.bounded(cartpole.lbu, us[:, i], cartpole.ubu))
            # opti.subject_to(cartpole.lbu <= us[:, i])
            # opti.subject_to(us[:, i] <= cartpole.ubu)
        
        self.opti = opti
        self.n_x = n_x
        self.n_u = n_u
        self.J = J
        self.xs = xs
        self.us = us
        self.x0 = x0

    def init(self, x0_val=None):
        if x0_val is None:
            x0_val = np.zeros(self.n_x)

        self.opti.set_value(self.x0, x0_val)

        # use IPOPT as NLP solver
        init_solver_option = {'print_time': False, 'calc_lam_x': True, 'calc_lam_p': True, 'ipopt': {'mu_min': 0.1, 'max_iter': 1000, 'warm_start_init_point': 'yes', 'print_level':0, 'print_timing_statistics':'no'}}
        self.opti.solver('ipopt', init_solver_option)

        # to get initial strict solution
        sol = self.opti.solve()

        # solver option for MPC
        self.solver_option = init_solver_option
        self.solver_option['ipopt']['max_iter'] = 10
        self.opti.solver('ipopt', self.solver_option)
        
        # store primal and dual variables for warm-start
        self.xs_opt = sol.value(self.xs)
        self.us_opt = sol.value(self.us)
        self.lam_gs_opt = sol.value(self.opti.lam_g)

    def solve(self, x0_val):
        # update current state
        self.opti.set_value(self.x0, x0_val)

        # warm start
        self.opti.set_initial(self.xs, self.xs_opt)
        self.opti.set_initial(self.us, self.us_opt)
        self.opti.set_initial(self.opti.lam_g, self.lam_gs_opt)

        try:
            sol = self.opti.solve()
        except:
            # print('fail')
            pass
        
        # store primal and dual variables for warm-start
        self.xs_opt = self.opti.debug.value(self.xs)
        self.us_opt = self.opti.debug.value(self.us)
        self.lam_gs_opt = self.opti.debug.value(self.opti.lam_g)

        # In MPC we use initial optimal input as actual input
        u0 = self.us_opt[0]
        if not isinstance(u0, np.ndarray):
            u0 = np.array([u0])

        return u0


### MPC simulation

In [ ]:
import math


mpc = MPC()
mpc.init()

# f(x, u)
f = CartPole(module='numpy').dynamics

# simulation condition
T_sim = 10.0
sampling_time = 0.010
N_sim = math.ceil(T_sim / sampling_time)
ts_sim = np.array([i*sampling_time for i in range(N_sim + 1)])

xs_sim = []
us_sim = []
x = np.zeros(n_x)
u = np.zeros(n_u)

# simulation
for t in ts_sim:
    if t.is_integer():
        print('t = ', t)

    u = mpc.solve(x)

    xs_sim.append(x)
    us_sim.append(u)

    x_next = x + f(x, u) * sampling_time
    x = x_next


In [ ]:
import path_ocpy
from ocpy.plotter import Plotter


xs_sim = np.array(xs_sim)
us_sim = np.array(us_sim)

log_dir = path_ocpy.dir + '../log/cartpole'

# print('final cost:', sol.value(J))

plotter = Plotter(log_dir, xs_sim, us_sim, ts_sim)
plotter.plot(save=False)
